### Word Embeddings - Pretrained Embeddings - GLOVE -LSTM

In [1]:
!unzip "glove.6B.100d.zip" -d "glove.6B.100d"

Archive:  glove.6B.100d.zip
  inflating: glove.6B.100d/glove.6B.100d.txt  


In [2]:
import pandas as pd

df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df = df.dropna()
df.shape

(18285, 5)

In [4]:
## Get Independent Features
X = df.drop('label',axis=1)

## Get Dependent Feature
y = df['label']

print(X.shape,y.shape) 

(18285, 4) (18285,)


### Text Preprocessing or Data Preprocessing

In [5]:
messages = X.copy()
messages.reset_index(inplace=True)
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [7]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import stopwords
ps = PorterStemmer()
wn = WordNetLemmatizer()
corpus = []

for i in range(0,len(messages)):
    ## Substitute all character except a-zA-Z
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [wn.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review) ### It will make a complete sentence , If not written words will be splitted in each sentence
    corpus.append(review)
    
corpus

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organizing action partner soros linked indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor pay 

In [9]:
len(corpus)

18285

### Pretrained Embeddings - Using GLOVE 

In [10]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

### Instead of One Hot Representation we will use Tokenizer here

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts(corpus)
vocab_size = len(t.word_index) + 1
print('Vocab Size : ' , vocab_size)
# integer encode the documents
encoded_docs = t.texts_to_sequences(corpus)
print(encoded_docs)

Vocab Size :  17491
[[29, 526, 558, 249, 136, 81, 449, 1734, 3154, 6801], [760, 6, 7, 87, 31, 716, 5], [293, 784, 34, 695], [507, 132, 1082, 8, 1975, 3519], [1111, 31, 2636, 4568, 9425, 221, 31, 6802, 110, 9426], [3520, 3521, 294, 127, 322, 4, 3972, 102, 284, 1412, 2101, 1217, 209, 14, 5], [6803, 6804, 50, 337, 2414, 106, 130, 1218, 1, 3, 2], [59, 1489, 47, 1005, 16, 6805, 4, 1976, 1, 3, 2], [11, 6806, 527, 2240, 229, 965, 9427, 2637, 4, 785], [1112, 1846, 5433, 141, 5434, 77, 625, 1330], [48, 2241, 2415, 190, 966, 206, 967, 4569, 3155, 3156, 256], [8, 90, 136, 559, 4, 16], [1977, 733, 83, 1847, 413, 32, 1490, 4570, 968], [450, 1331, 1332, 5435, 88, 107, 1333, 1, 3, 2], [786, 1413, 155, 5436, 329, 1, 3, 2], [929, 677, 243, 60, 268, 930, 279, 346, 414, 280], [25, 374, 6], [1735, 1848, 2102, 9, 4, 147, 849, 5], [4571, 9428, 7, 172, 230, 236, 21, 191, 221], [4572, 9429, 4, 1978, 9430, 5], [560, 1979, 103, 393, 2638, 21, 244, 415, 787, 1, 3, 2], [2863, 138, 1414, 1736, 91, 179, 430], [209,

### Using Pad Sequences

In [12]:
max_length = max([len(sen.split(' ')) for sen in corpus ])
max_length

47

In [13]:
# pad documents to a max length of 4 words
max_length = max([len(sen.split(' ')) for sen in corpus ])
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[  29  526  558 ...    0    0    0]
 [ 760    6    7 ...    0    0    0]
 [ 293  784   34 ...    0    0    0]
 ...
 [6585  348 2352 ...    0    0    0]
 [ 439   16  683 ...    0    0    0]
 [ 239   96 1410 ...    0    0    0]]


In [14]:
padded_docs.shape

(18285, 47)

### Load Glove Word Embedding File as Dictionary of Word to Embedding array of Features

In [15]:
# load the whole embedding into memory
from numpy import asarray
embeddings_index = dict()
f = open('glove.6B.100d/glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
print(type(embeddings_index))

Loaded 400000 word vectors.
<class 'dict'>


In [16]:
embeddings_index['house']  ### there will be 100 d of word 'house'

array([-0.18867  , -0.040943 ,  0.58878  ,  0.11062  ,  0.14236  ,
        0.4885   , -0.31832  ,  0.53819  , -0.018549 ,  0.029687 ,
        0.30299  , -0.16522  , -0.18896  ,  0.5148   , -0.79405  ,
        0.26409  ,  0.027747 ,  0.041163 , -0.49378  , -0.14263  ,
        0.29017  , -0.25369  ,  0.70559  , -1.0501   , -0.49344  ,
       -0.37148  , -0.85796  , -0.55158  , -0.60251  , -0.0099676,
        0.8725   ,  0.12149  ,  0.551    ,  0.49924  , -0.3088   ,
        1.1067   , -0.15494  , -0.29923  ,  0.91149  ,  0.19859  ,
       -0.73946  , -1.0182   ,  0.37208  , -0.10043  ,  0.13537  ,
       -0.52687  , -0.60437  , -0.15906  ,  0.49283  , -0.61386  ,
        0.046815 , -0.88806  ,  0.60229  ,  0.72199  , -0.4316   ,
       -3.0706   , -0.11233  , -0.45713  ,  0.95737  ,  0.59174  ,
       -0.17124  ,  0.65746  ,  0.44741  ,  0.6101   ,  1.0216   ,
       -0.2458   ,  0.90191  ,  0.78319  ,  0.28272  , -0.4539   ,
        0.16309  , -0.0078932, -0.27714  , -0.87249  , -0.1971

In [17]:
vocab_size

17491

In [18]:
t.word_index.items() ### Each word has its own Integer value which is required before Embedding layer

dict_items([('new', 1), ('time', 2), ('york', 3), ('trump', 4), ('breitbart', 5), ('hillary', 6), ('clinton', 7), ('u', 8), ('donald', 9), ('election', 10), ('obama', 11), ('say', 12), ('state', 13), ('video', 14), ('war', 15), ('russia', 16), ('president', 17), ('comment', 18), ('world', 19), ('year', 20), ('american', 21), ('day', 22), ('news', 23), ('email', 24), ('fbi', 25), ('attack', 26), ('america', 27), ('report', 28), ('house', 29), ('white', 30), ('woman', 31), ('medium', 32), ('police', 33), ('get', 34), ('campaign', 35), ('vote', 36), ('call', 37), ('bill', 38), ('first', 39), ('one', 40), ('briefing', 41), ('black', 42), ('syria', 43), ('anti', 44), ('show', 45), ('right', 46), ('plan', 47), ('russian', 48), ('people', 49), ('win', 50), ('democrat', 51), ('life', 52), ('watch', 53), ('may', 54), ('make', 55), ('man', 56), ('go', 57), ('muslim', 58), ('back', 59), ('million', 60), ('could', 61), ('china', 62), ('child', 63), ('like', 64), ('putin', 65), ('n', 66), ('wikilea

###  Creating Embedding Matrix with Glove Weights

Next, we need to create a matrix of one embedding for each word in the training dataset. We can do that by enumerating all unique words in the Tokenizer.word_index and locating the embedding weight vector from the loaded GloVe embedding.

In [19]:
from numpy import zeros
embedding_matrix = zeros((vocab_size, 100))
print(embedding_matrix)
embedding_matrix.shape

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


(17491, 100)

In [20]:
embedding_vector = embeddings_index.get('house')
embedding_vector

array([-0.18867  , -0.040943 ,  0.58878  ,  0.11062  ,  0.14236  ,
        0.4885   , -0.31832  ,  0.53819  , -0.018549 ,  0.029687 ,
        0.30299  , -0.16522  , -0.18896  ,  0.5148   , -0.79405  ,
        0.26409  ,  0.027747 ,  0.041163 , -0.49378  , -0.14263  ,
        0.29017  , -0.25369  ,  0.70559  , -1.0501   , -0.49344  ,
       -0.37148  , -0.85796  , -0.55158  , -0.60251  , -0.0099676,
        0.8725   ,  0.12149  ,  0.551    ,  0.49924  , -0.3088   ,
        1.1067   , -0.15494  , -0.29923  ,  0.91149  ,  0.19859  ,
       -0.73946  , -1.0182   ,  0.37208  , -0.10043  ,  0.13537  ,
       -0.52687  , -0.60437  , -0.15906  ,  0.49283  , -0.61386  ,
        0.046815 , -0.88806  ,  0.60229  ,  0.72199  , -0.4316   ,
       -3.0706   , -0.11233  , -0.45713  ,  0.95737  ,  0.59174  ,
       -0.17124  ,  0.65746  ,  0.44741  ,  0.6101   ,  1.0216   ,
       -0.2458   ,  0.90191  ,  0.78319  ,  0.28272  , -0.4539   ,
        0.16309  , -0.0078932, -0.27714  , -0.87249  , -0.1971

Now above embedded vector of 'house' will get replaced in Main Embedding matrix

In [21]:
### Above process will be done for each and every word . Its value will get stored in Embedded_Matrix

for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_matrix

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-4.39589992e-02,  1.89359993e-01,  6.61099970e-01, ...,
        -1.41680002e-01,  9.27890003e-01,  5.90579987e-01],
       [-2.42209993e-02, -3.48550007e-02,  3.57100010e-01, ...,
        -8.75680000e-02,  2.59609997e-01,  5.07830009e-02],
       ...,
       [ 5.88590018e-02,  4.81730014e-01,  2.40830004e-01, ...,
         1.27770007e-01,  3.25199991e-01, -1.94220006e-01],
       [ 5.81210017e-01,  8.69250000e-01, -5.72660029e-01, ...,
         1.08290002e-01, -8.74859989e-01, -2.51379997e-01],
       [-8.36940035e-02, -1.61419995e-02,  2.59629991e-02, ...,
        -9.49140012e-05, -3.57239991e-01, -9.09620002e-02]])

### Embedding Layer

Now we will Directly provide the Embedded Matrix to the Embedding Layer which has weights from Glove

The key difference is that the embedding layer can be seeded with the GloVe word embedding weights. We chose the 100-dimensional version, therefore the Embedding layer must be defined with output_dim set to 100. Finally, we do not want to update the learned word weights in this model, therefore we will set the trainable attribute for the model to be False.

Here Learning will not be done , Becuase we have alreadY used pretrained glove embedding

In [22]:
max_length

47

In [23]:
embedding_matrix.shape

(17491, 100)

### In Embedding layer we will set trainable=False , Because we dont need to train over here as we are already providing trainable embedded matrix

In [24]:
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)

###  Creating Model

In [25]:
model = Sequential()
model.add(e)
model.add(LSTM(100))  ### 1 LSTM layer with 100 Neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 47, 100)           1749100   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 1,829,601
Trainable params: 80,501
Non-trainable params: 1,749,100
_________________________________________________________________
None


In [26]:
padded_docs[0]

array([  29,  526,  558,  249,  136,   81,  449, 1734, 3154, 6801,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0], dtype=int32)

In [27]:
padded_docs.shape

(18285, 47)

In [28]:
import numpy as np

X_final = np.array(padded_docs)
y_final = np.array(y)

X_final.shape,y_final.shape

((18285, 47), (18285,))

### Train Test Split

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((12250, 47), (12250,), (6035, 47), (6035,))

### Model Training

In [30]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 10ms/step - loss: 0.4506 - accuracy: 0.7594 - val_loss: 0.2714 - val_accuracy: 0.8963
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.2468 - accuracy: 0.9011 - val_loss: 0.2320 - val_accuracy: 0.8938
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.2348 - accuracy: 0.9014 - val_loss: 0.2423 - val_accuracy: 0.8941
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.2127 - accuracy: 0.9064 - val_loss: 0.2088 - val_accuracy: 0.9100
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1702 - accuracy: 0.9283 - val_loss: 0.2010 - val_accuracy: 0.9198
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1689 - accuracy: 0.9269 - val_loss: 0.2014 - val_accuracy: 0.9110
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1329 - accuracy: 0.9466 - val_loss: 0.2130 - val_accuracy: 0.9231
Epoc

### Performance metrics

In [31]:
y_pred=model.predict_classes(X_test)

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.9260977630488815
[[3055  364]
 [  82 2534]]
              precision    recall  f1-score   support

           0       0.97      0.89      0.93      3419
           1       0.87      0.97      0.92      2616

    accuracy                           0.93      6035
   macro avg       0.92      0.93      0.93      6035
weighted avg       0.93      0.93      0.93      6035



### Simple RNN with Glove

In [32]:
from tensorflow.keras.layers import Dropout
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)

model0 = Sequential()
model0.add(e)
model0.add(Flatten())
model0.add(Dense(1, activation='sigmoid'))
# compile the model
model0.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model0.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 47, 100)           1749100   
_________________________________________________________________
flatten (Flatten)            (None, 4700)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 4701      
Total params: 1,753,801
Trainable params: 4,701
Non-trainable params: 1,749,100
_________________________________________________________________
None


In [33]:
model0.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 1s 4ms/step - loss: 0.5122 - accuracy: 0.7344 - val_loss: 0.3512 - val_accuracy: 0.8522
Epoch 2/10
192/192 [==============================] - 1s 4ms/step - loss: 0.3310 - accuracy: 0.8625 - val_loss: 0.3063 - val_accuracy: 0.8713
Epoch 3/10
192/192 [==============================] - 1s 3ms/step - loss: 0.2862 - accuracy: 0.8804 - val_loss: 0.2890 - val_accuracy: 0.8789
Epoch 4/10
192/192 [==============================] - 1s 3ms/step - loss: 0.2691 - accuracy: 0.8886 - val_loss: 0.2799 - val_accuracy: 0.8824
Epoch 5/10
192/192 [==============================] - 1s 4ms/step - loss: 0.2464 - accuracy: 0.9010 - val_loss: 0.2746 - val_accuracy: 0.8832
Epoch 6/10
192/192 [==============================] - 1s 4ms/step - loss: 0.2354 - accuracy: 0.9081 - val_loss: 0.2716 - val_accuracy: 0.8848
Epoch 7/10
192/192 [==============================] - 1s 4ms/step - loss: 0.2253 - accuracy: 0.9116 - val_loss: 0.2702 - val_accuracy: 0.8847
Epoch 

In [34]:
y_pred=model0.predict_classes(X_test)

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.8869925434962718
[[3010  409]
 [ 273 2343]]
              precision    recall  f1-score   support

           0       0.92      0.88      0.90      3419
           1       0.85      0.90      0.87      2616

    accuracy                           0.89      6035
   macro avg       0.88      0.89      0.89      6035
weighted avg       0.89      0.89      0.89      6035



### Adding Dropout layer with LSTM

In [35]:
from tensorflow.keras.layers import Dropout
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)

model1 = Sequential()
model1.add(e)
model1.add(Dropout(0.3))
model1.add(LSTM(100))  ### 1 LSTM layer with 100 Neurons
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 47, 100)           1749100   
_________________________________________________________________
dropout (Dropout)            (None, 47, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,829,601
Trainable params: 80,501
Non-trainable params: 1,749,100
_________________________________________________________________
None


In [36]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 9ms/step - loss: 0.4625 - accuracy: 0.7451 - val_loss: 0.2626 - val_accuracy: 0.8966
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.2638 - accuracy: 0.8966 - val_loss: 0.2268 - val_accuracy: 0.8961
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.2270 - accuracy: 0.8995 - val_loss: 0.2183 - val_accuracy: 0.8991
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.2184 - accuracy: 0.9033 - val_loss: 0.2267 - val_accuracy: 0.9064
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.2538 - accuracy: 0.8957 - val_loss: 0.2214 - val_accuracy: 0.8973
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1967 - accuracy: 0.9118 - val_loss: 0.2018 - val_accuracy: 0.9112
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1937 - accuracy: 0.9136 - val_loss: 0.2025 - val_accuracy: 0.9183
Epoch 

In [37]:
y_pred=model1.predict_classes(X_test)

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.91996685998343
[[3196  223]
 [ 260 2356]]
              precision    recall  f1-score   support

           0       0.92      0.93      0.93      3419
           1       0.91      0.90      0.91      2616

    accuracy                           0.92      6035
   macro avg       0.92      0.92      0.92      6035
weighted avg       0.92      0.92      0.92      6035



### Using Bidirectional LSTM

In [39]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)

model2 = Sequential()
model2.add(e)
model2.add(Bidirectional(LSTM(100)))  ### 1 LSTM layer with 100 Neurons
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model2.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 47, 100)           1749100   
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 1,910,101
Trainable params: 161,001
Non-trainable params: 1,749,100
_________________________________________________________________
None


In [40]:
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 14ms/step - loss: 0.3954 - accuracy: 0.7993 - val_loss: 0.2233 - val_accuracy: 0.9022
Epoch 2/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1994 - accuracy: 0.9153 - val_loss: 0.1987 - val_accuracy: 0.9090
Epoch 3/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1695 - accuracy: 0.9264 - val_loss: 0.1841 - val_accuracy: 0.9203
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1527 - accuracy: 0.9335 - val_loss: 0.1869 - val_accuracy: 0.9225
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1323 - accuracy: 0.9476 - val_loss: 0.1719 - val_accuracy: 0.9283
Epoch 6/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1107 - accuracy: 0.9571 - val_loss: 0.1807 - val_accuracy: 0.9256
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1005 - accuracy: 0.9588 - val_loss: 0.2138 - val_accuracy: 0.9163

In [41]:
y_pred=model2.predict_classes(X_test)

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.9201325600662801
[[3226  193]
 [ 289 2327]]
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      3419
           1       0.92      0.89      0.91      2616

    accuracy                           0.92      6035
   macro avg       0.92      0.92      0.92      6035
weighted avg       0.92      0.92      0.92      6035



### Bidirectional LSTM and Dropout layer

In [42]:
from tensorflow.keras.layers import Dropout
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)

model3 = Sequential()
model3.add(e)
model3.add(Dropout(0.3))
model3.add(Bidirectional(LSTM(100)))  ### 1 LSTM layer with 100 Neurons
model3.add(Dense(1,activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model3.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 47, 100)           1749100   
_________________________________________________________________
dropout_1 (Dropout)          (None, 47, 100)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 201       
Total params: 1,910,101
Trainable params: 161,001
Non-trainable params: 1,749,100
_________________________________________________________________
None


In [43]:
model3.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 14ms/step - loss: 0.4067 - accuracy: 0.7897 - val_loss: 0.2142 - val_accuracy: 0.9031
Epoch 2/10
192/192 [==============================] - 2s 10ms/step - loss: 0.2125 - accuracy: 0.9069 - val_loss: 0.2058 - val_accuracy: 0.9105
Epoch 3/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1967 - accuracy: 0.9160 - val_loss: 0.1891 - val_accuracy: 0.9163
Epoch 4/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1717 - accuracy: 0.9239 - val_loss: 0.1846 - val_accuracy: 0.9193
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1612 - accuracy: 0.9323 - val_loss: 0.1822 - val_accuracy: 0.9223
Epoch 6/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1530 - accuracy: 0.9348 - val_loss: 0.1833 - val_accuracy: 0.9234
Epoch 7/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1387 - accuracy: 0.9415 - val_loss: 0.1827 - val_accuracy: 0.9218

In [44]:
y_pred=model3.predict_classes(X_test)

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.9272576636288318
[[3080  339]
 [ 100 2516]]
              precision    recall  f1-score   support

           0       0.97      0.90      0.93      3419
           1       0.88      0.96      0.92      2616

    accuracy                           0.93      6035
   macro avg       0.92      0.93      0.93      6035
weighted avg       0.93      0.93      0.93      6035



### 2 LSTM - Dropout - Glove 

In [51]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)

model4 = Sequential()
model4.add(e)
model4.add(Dropout(0.3))
model4.add(LSTM(100,return_sequences=True))  ### 1 LSTM layer with 100 Neurons
model4.add(Dropout(0.3))
model4.add(LSTM(100))
model4.add(Dense(1,activation='sigmoid'))
model4.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model4.summary())

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 47, 100)           1749100   
_________________________________________________________________
dropout_14 (Dropout)         (None, 47, 100)           0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 47, 100)           80400     
_________________________________________________________________
dropout_15 (Dropout)         (None, 47, 100)           0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 1,910,001
Trainable params: 160,901
Non-trainable params: 1,749,100
_____________________________________

In [52]:
model4.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 14ms/step - loss: 0.4286 - accuracy: 0.7753 - val_loss: 0.2644 - val_accuracy: 0.8948
Epoch 2/10
192/192 [==============================] - 2s 10ms/step - loss: 0.2479 - accuracy: 0.9023 - val_loss: 0.2234 - val_accuracy: 0.8973
Epoch 3/10
192/192 [==============================] - 2s 10ms/step - loss: 0.2279 - accuracy: 0.9030 - val_loss: 0.2278 - val_accuracy: 0.9047
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.2080 - accuracy: 0.9070 - val_loss: 0.1992 - val_accuracy: 0.9084
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1998 - accuracy: 0.9134 - val_loss: 0.2123 - val_accuracy: 0.9147
Epoch 6/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1724 - accuracy: 0.9250 - val_loss: 0.1903 - val_accuracy: 0.9183
Epoch 7/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1655 - accuracy: 0.9272 - val_loss: 0.1883 - val_accuracy: 0.9195

In [53]:
y_pred=model4.predict_classes(X_test)

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.9300745650372825
[[3152  267]
 [ 155 2461]]
              precision    recall  f1-score   support

           0       0.95      0.92      0.94      3419
           1       0.90      0.94      0.92      2616

    accuracy                           0.93      6035
   macro avg       0.93      0.93      0.93      6035
weighted avg       0.93      0.93      0.93      6035



### Stacked Bidirectional LSTM - Dropout - Glove

In [56]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)

model5 = Sequential()
model5.add(e)
model5.add(Dropout(0.3))
model5.add(Bidirectional(LSTM(128, recurrent_dropout=0.4, return_sequences=True)))
model5.add(Dropout(0.3))
model5.add(Bidirectional(LSTM(128, recurrent_dropout=0.4)))
model5.add(Dense(1,activation='sigmoid'))
model5.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model5.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 47, 100)           1749100   
_________________________________________________________________
dropout_18 (Dropout)         (None, 47, 100)           0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 47, 256)           234496    
_________________________________________________________________
dropout_19 (Dropout)         (None, 47, 256)           0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 256)               394240    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 2,378,093
Trainable params: 628,993
Non-trainable params: 1,749,100
_____________________________________

In [57]:
model5.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 127s 632ms/step - loss: 0.3738 - accuracy: 0.8167 - val_loss: 0.2214 - val_accuracy: 0.9026
Epoch 2/10
192/192 [==============================] - 116s 605ms/step - loss: 0.2123 - accuracy: 0.9063 - val_loss: 0.2013 - val_accuracy: 0.9087
Epoch 3/10
192/192 [==============================] - 118s 617ms/step - loss: 0.1873 - accuracy: 0.9219 - val_loss: 0.1812 - val_accuracy: 0.9188
Epoch 4/10
192/192 [==============================] - 117s 610ms/step - loss: 0.1742 - accuracy: 0.9230 - val_loss: 0.1832 - val_accuracy: 0.9190
Epoch 5/10
192/192 [==============================] - 116s 603ms/step - loss: 0.1599 - accuracy: 0.9335 - val_loss: 0.1769 - val_accuracy: 0.9246
Epoch 6/10
192/192 [==============================] - 118s 615ms/step - loss: 0.1488 - accuracy: 0.9382 - val_loss: 0.1739 - val_accuracy: 0.9238
Epoch 7/10
192/192 [==============================] - 116s 607ms/step - loss: 0.1432 - accuracy: 0.9408 - val_loss: 0.1733 -

In [58]:
y_pred=model5.predict_classes(X_test)

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.9292460646230323
[[3072  347]
 [  80 2536]]
              precision    recall  f1-score   support

           0       0.97      0.90      0.94      3419
           1       0.88      0.97      0.92      2616

    accuracy                           0.93      6035
   macro avg       0.93      0.93      0.93      6035
weighted avg       0.93      0.93      0.93      6035

